In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from topostats.io import LoadScans

import perovskite_flattening as perov_flatten

from scipy.signal import convolve2d

In [ ]:
# Set data directory
# DATA_DIR = Path("/Users/sylvi/topo_data/perovskite/perovskite_images/Me/")
DATA_DIR = Path("/Users/sylvi/topo_data/perovstats_paper/raw_data/")
FILE_EXT = ".spm"

files = list(DATA_DIR.glob(f"*{FILE_EXT}"))
files.sort()
for file in files:
    print(file)

# directory_contents = DATA_DIR.iterdir()
# for item in directory_contents:
#     print(item)

In [ ]:
for file in files:
    print(file.stem)

    loadscans = LoadScans([file], channel="Height")
    loadscans.get_data()

    img_dict = loadscans.img_dict
    image = img_dict[file.stem]["image_original"]

    # Set the 99th percentile as the maximum value, to remove outliers
    image[image > np.percentile(image, 99)] = np.percentile(image, 99)
    # Set the 1st percentile as the minimum value, to remove outliers
    image[image < np.percentile(image, 1)] = np.percentile(image, 1)

    plt.hist(image.flatten(), bins=1000)
    plt.show()

    plt.imshow(image, cmap="viridis")
    plt.show()

    plt.imsave(DATA_DIR / f"{file.stem}.png", image, cmap="viridis")

In [ ]:
# Load standard height data from images
loadscans = LoadScans(files, channel="Height Sensor")
loadscans.get_data()

img_dict = loadscans.img_dict

for filename, data in img_dict.items():
    print(filename)

    image = data["image_original"]

    plt.imshow(image)
    plt.show()

    # Apply an edge finding convolution filter

    # Create an edge finding kernel
    kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])

    edges = convolve2d(image, kernel, mode="same")

    # print(f"min: {np.min(edges)} max: {np.max(edges)}")
    # # Plot histogram of the edge values
    # plt.hist(edges.flatten(), bins=1000)
    # plt.show()

    # Limit the range of the edge values to the 99th percentile
    edges[edges > np.percentile(edges, 99)] = np.percentile(edges, 99)
    # Limit the range of the edge values to the 1st percentile
    edges[edges < np.percentile(edges, 1)] = np.percentile(edges, 1)

    plt.imshow(edges, cmap="viridis")
    plt.show()

    plt.imsave(DATA_DIR / f"convolution_{filename}.png", edges, cmap="viridis")

In [ ]:
loadscans = LoadScans(files, channel="Height")
loadscans.get_data()

file_dict = loadscans.img_dict
for file, data in file_dict.items():
    img = data["image_original"]
    plt.imshow(img)
    plt.show()

In [ ]:
# Try out colourmaps

file = Path("/Users/sylvi/topo_data/perovskite/perovskite_images/Me/20230207_Me4_NPs_perov_5um.0_00006.spm")

loadscans = LoadScans([file], "Height Sensor")
loadscans.get_data()
image = loadscans.image
px_to_nm_scaling = loadscans.pixel_to_nm_scaling

# Flatten
flattened = perov_flatten.flatten_image(image=image, order=3, plot_steps=False)

for cmap in [
    "viridis",
    "magma",
    "plasma",
    "inferno",
    "turbo",
    "cividis",
    "afmhot",
    "PiYG",
    "BrBG",
    "coolwarm",
    "hsv",
    "twilight",
    "twilight_shifted",
    "flag",
    "prism",
    "gist_ncar",
    "terrain",
]:
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(flattened, cmap=cmap)
    ax.set_title(cmap)
    plt.show()

In [ ]:
# Find all files of the specified file type
files = DATA_DIR.glob(f"*{FILE_EXT}")

for file in files:
    print(file)
    loadscans = LoadScans([file], "Height Sensor")
    loadscans.get_data()
    image = loadscans.image
    px_to_nm_scaling = loadscans.pixel_to_nm_scaling
    # Plot initial image
    # fig, ax = plt.subplots(figsize=(8, 8))
    # ax.imshow(image)
    # fig.savefig(str(file) + ".png")
    # plt.show()
    # Flatten
    flattened = perov_flatten.flatten_image(image=image, order=3, plot_steps=False)
    # fig, ax = plt.subplots(figsize=(8, 8))
    # ax.imshow(flattened)

    # # Save figure with axes
    # fig.savefig(str(file) + "_image.png")
    # plt.show()

    plt.imsave(str(file) + "_image.png", flattened, cmap="viridis")
    np.save(str(file) + "_image.npy", flattened)